## **Train the model on the Google AI Platform**

In [1]:
## Mount on google drive
from google.colab import drive
drive.mount('/content/drive/')
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()
## to the work directorty
import os
os.chdir("/content/drive/My Drive/Earth-Engine-with-Deep-Learning/trainer")

Mounted at /content/drive/


#### <font color=red> **！！Note: the runtime-version setting (supported tensorflow version in AI platform) should be consistency with the Tensorflow version in the colab, and we use Tensorflow 2.2.0 here!**

In [3]:
# !pip install tensorflow==2.2.0
import tensorflow as tf
print(tf.__version__)


2.2.0


In [4]:
import time
from ai_platform_package import config
from ai_platform_package import model
from ai_platform_package import dataLoader

Job_Name = 'impervious_unet_training_job_' + time.strftime("%Y%m%d%H%M")
Train_Package_Path = 'ai_platform_package'
Main_Trainer_Module = 'ai_platform_package.trainingTask'
Region = 'asia-east1'
Bucket = 'earth-engine-bucket-1'


gs://earth-engine-bucket-1/MSMT_RF_impervious_traData/Train_Landsat7*.tfrecord.gz


## <font color=red>!Carefully check the configuration(**config.py**) and training modules (**trainingTask**)</font>

<font color=red>**Note: if the training only exit in google drive, we need to copy the training from google drive to google cloud storage.**</font>

In [ ]:
# ## Copy the data from google drive to google cloud storage
# jsonFile_gs_dir_tra = 'gs://' + Bucket+ '/MSMT_RF_impervious_traData/'
# jsonFile_gs_dir_eva = 'gs://' + Bucket+ '/MSMT_RF_impervious_evaData/'
# !gsutil -m cp -r '/content/drive/My Drive/EE_Image/MSMT_RF_Impervious_traData/*.tfrecord.gz' {jsonFile_gs_dir_tra}
# !gsutil -m cp -r '/content/drive/My Drive/EE_Image/MSMT_RF_Impervious_evaData/*.tfrecord.gz' {jsonFile_gs_dir_eva}

### Check the data and the model

In [6]:
## Check the training data
# !cat ai_platform_package/config.py
# !cat ai_platform_package/dataLoader.py
# traData = dataLoader.get_training_dataset()  # the dataset for training 
# one_traBatch = iter(traData.take(1)).next()
# one_traBatch
# evaData = dataLoader.get_eval_dataset()  # the dataset for training 
# one_evaBatch = iter(evaData.take(1)).next()
# one_evaBatch
## check the model
# !cat ai_platform_package/trainingTask.py
# model = model.UNet(input_shape=(config.Kernel_shape[0], config.Kernel_shape[1], 6), nclasses=2) # the model for training
# print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 6) 0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 128, 128, 32) 11264       input_1[0][0]                    
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 64, 64, 64)   55936       sequential[0][0]                 
__________________________________________________________________________________________________
sequential_2 (Sequential)       (None, 32, 32, 64)   74368       sequential_1[0][0]               
______________________________________________________________________________________________

## Submit the package to AI Platform for training

In [ ]:
!gcloud ai-platform jobs submit training {Job_Name} \
    --job-dir {config.Job_Dir} \
    --package-path {Train_Package_Path} \
    --module-name {Main_Trainer_Module} \
    --region {Region} \
    --project {config.Project} \
    --runtime-version 2.2 \
    --python-version 3.7 \
    --scale-tier basic-gpu


In [9]:
desc = !gcloud ai-platform jobs describe {Job_Name} --project {config.Project}
state = desc.grep('state:')[0].split(':')[1].strip()
print(state)


RUNNING


## Inspect the trained model

In [ ]:
%load_ext tensorboard
%tensorboard --logdir {config.Logs_Dir}
# Logs_Dir = 'gs://earth-engine-bucket-1/ai_platform_train/unet_384_v1/logs'
# %tensorboard --logdir {Logs_Dir}